**NOTE**: Here we test the performance of the STged with gene expression deconvolution methods

In [1]:

#remove(list = ls())
#define the conda env

library(reticulate)
use_condaenv("ccnet", required = TRUE)

python_env = "python_env"

In [2]:
library(ggplot2)
library(scatterpie)
library(RColorBrewer)
library(TCA)
library(ENIGMA)

Package 'ENIGMA' version 0.1.6
ENIGMA is built for fully deconvolution.



In [3]:
# Source code
source("./code/STged.R")
source("./code/benchmark.R")

In [4]:
colors <- colorRampPalette(brewer.pal(9, "Set3"))(12)


# Load data the generated simulated data
##  we use the FISH+ data from the mouse tissue. There are 69 spots.

In [5]:
sim_data = readRDS("./realdata/FISH+/sim_fishplus_input.RDS")

TRUE_F = sim_data$trueF
beta.type = sim_data$beta.type

cell_type = colnames(beta.type)
cell_type

[1] "astrocytes" "endo_mural" "eNeuron"    "iNeuron"    "microglia" 
[6] "Olig"

# Run the STged step by step

## Step 1: clear data

In [6]:
depthscale = 1e6
clean.only = FALSE
datax = data_process(sc_exp = sim_data$sc_exp,   sc_label = sim_data$sc_label, 
                     spot_exp = sim_data$spot_exp,  spot_loc = sim_data$spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0.05, 
                     expression_threshold = 0,
                     nUMI =  100, verbose = FALSE, clean.only = clean.only)


In [7]:
# use for the other completed methods
datax_count = data_process(sc_exp = sim_data$sc_exp,   sc_label = sim_data$sc_label, 
                     spot_exp = sim_data$spot_exp,  spot_loc = sim_data$spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0.05, 
                     expression_threshold = 0,
                     nUMI =  100, verbose = FALSE, clean.only = TRUE)

##  True cell type-specific gene expression data

In [8]:
gene_sec = rownames(datax$sc_exp)

for(i in 1:length(cell_type)){

  TRUE_F[[i]] = TRUE_F[[i]][gene_sec,colnames(datax$spot_exp )]

}


saveRDS(TRUE_F, file ="./results/FISH+/True_cell_exp.RDS")

## Step 2: construct spatial correlation structures

In [9]:
cat("Construct spatial correlation", "\n")
L.mat = dis_weight(spot_loc = datax$spot_loc, spot_exp = datax$spot_exp, k = 4, 
                   quantile_prob_bandwidth = 1/3, method = "Square", 
                   coord_type = "grid")

Construct spatial correlation 


## Step 3: construct reference gene matrix

In [10]:

cat("Construct reference gene matrix", "\n")

ref_exp = create_group_exp(sc_exp = datax$sc_exp, sc_label = datax$sc_label)


ref_exp = ref_exp[rownames(datax$spot_exp), cell_type]
colnames(ref_exp )

beta.type = beta.type[colnames(datax$spot_exp),]


Construct reference gene matrix 


[1] "astrocytes" "endo_mural" "eNeuron"    "iNeuron"    "microglia" 
[6] "Olig"

In [11]:
p = nrow(datax$spot_exp)
p
K = ncol(beta.type)
beta.ind.temp = beta.type>0.05
beta.ind =  matrix(list(), K ,1)
for (i in 1:K){
  
  beta.ind[[i]] =  matrix( rep(beta.ind.temp[,i],p), nrow = p, byrow = TRUE)
}


saveRDS(beta.ind, file ="./results/FISH+/cell_prop_indx.RDS")

[1] 6471

## Step 4: run the main model
### Step 4-1: run the main model with true cell type proportion

In [12]:
source("./code/STged.R")


In [13]:

start_time <- Sys.time()

if (!file.exists("./results/FISH+/True_stged.RDS")) {
  stged.est.true <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.true, file = "./results/FISH+/True_stged.RDS")
} else {

  stged.est.true <- readRDS("./results/FISH+/True_stged.RDS")
}

end_time <- Sys.time()


reg1 <- TRUE

if (reg1 && !file.exists("./results/FISH+/True_stged_reg1.RDS")) {
  stged.est.reg1 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg1, file = "./results/FISH+/True_stged_reg1.RDS")
} else {

  stged.est.reg1 <- readRDS("./results/FISH+/True_stged_reg1.RDS")
}


reg2 <- TRUE
if (reg2 && !file.exists("./results/FISH+/True_stged_reg2.RDS")) {
  stged.est.reg2 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )

  saveRDS(stged.est.reg2, file = "./results/FISH+/True_stged_reg2.RDS")
} else {
 
  stged.est.reg2 <- readRDS("./results/FISH+/True_stged_reg2.RDS")
}



reg3 <- TRUE
if (reg3 && !file.exists("./results/FISH+/True_stged_reg3.RDS")) {
  stged.est.reg3 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )

  saveRDS(stged.est.reg3, file = "./results/FISH+/True_stged_reg3.RDS")
} else {
 
  stged.est.reg3 <- readRDS("./results/FISH+/True_stged_reg3.RDS")
}


We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.03392335 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 


We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.03392335 
Select value of lambda2 0 
Run the main algorithm... 


Select value of lambda1 0 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 


Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 


In [14]:
# the optial values                      
lambda1_opt = stged.est.true$lambda1
lambda2_opt = stged.est.true$lambda2

# Run the benchmarking methods

In [15]:
exp_lsr = st_mu_est(srt_exp = datax$spot_exp,  beta= beta.type)

exp_sc = sc_mu_est( sc_mu = ref_exp, beta = beta.type)

exp_rctd =  RCTDexp( srt_exp = datax$spot_exp, ref_mu = ref_exp,beta = beta.type )

exp_spotdecon = spotdecon_est(srt_exp = datax$spot_exp,  beta= beta.type)


In [16]:
# Check and save exp_ENIGMA result
if (!file.exists("./results/FISH+/True_exp_ENIGMA.RDS")) {
  exp_ENIGMA <- ENIGMA_est(
    spot_exp = datax_count$spot_exp, 
    sc_exp = datax_count$sc_exp, 
    sc_label = datax_count$sc_label, 
    beta = beta.type )
  
  # Save the result if computation is performed
  saveRDS(exp_ENIGMA, file = "./results/FISH+/True_exp_ENIGMA.RDS")
} else {
  # Load the result if it already exists
  exp_ENIGMA <- readRDS("./results/FISH+/True_exp_ENIGMA.RDS")
}


In [17]:
# Check and save exp_TCA result
if (!file.exists("./results/FISH+/True_exp_TCA.RDS")) {
  
 exp_TCA <- TCA_est(spot_exp  = datax_count$spot_exp, beta= beta.type)
  
  # Save the result if computation is performed
  saveRDS(exp_TCA, file = "./results/FISH+/True_exp_TCA.RDS")
} else {
  # Load the result if it already exists
  exp_TCA <- readRDS("./results/FISH+/True_exp_TCA.RDS")
}


In [18]:
est_est= list(STged = stged.est.true$V.hat,
              STged.reg1 = stged.est.reg1$V.hat,
              STged.reg2 =stged.est.reg2$V.hat,
              STged.reg3 =stged.est.reg3$V.hat,
              RCTD = exp_rctd,
              TCA = exp_TCA,
              ENIGMA = exp_ENIGMA,
              ref_mu = exp_sc,
              LSR = exp_lsr,
              Spotdecon = exp_spotdecon)

files = paste0("./results/FISH+/True_spot_decon.RDS")

saveRDS(est_est, file =files)

# run with model with estimated cell type proportion

In [19]:
## cell type proportion predected from the proposed cell type deconvolution methods
res_decon_ct = readRDS("./realdata/FISH+/EnDecon_major/FISH_Results.Deconv.RDS")
names(res_decon_ct)

[1] "CARD"          "RCTD"          "SpatialDWLS"   "SPOTlight"    
[5] "Stereoscope"   "cell2location" "EnDecon"       "DWLS"         
[9] "SONAR"

In [20]:
beta.est = res_decon_ct$EnDecon
beta.type.RCTD = res_decon_ct$RCTD
## use the esimated cell type proprotion

In [21]:
## use the esimated cell type proprotion
beta.type = beta.est

In [22]:
 
start_time <- Sys.time()

if (!file.exists("./results/FISH+/EST_stged.RDS")) {
  stged.est.true <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.true, file = "./results/FISH+/EST_stged.RDS")
} else {

  stged.est.true <- readRDS("./results/FISH+/EST_stged.RDS")
}

end_time <- Sys.time()


reg1 <- TRUE

if (reg1 && !file.exists("./results/FISH+/EST_stged_reg1.RDS")) {
  stged.est.reg1 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = NULL, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )
  
  saveRDS(stged.est.reg1, file = "./results/FISH+/EST_stged_reg1.RDS")
} else {

  stged.est.reg1 <- readRDS("./results/FISH+/EST_stged_reg1.RDS")
}


reg2 <- TRUE
if (reg2 && !file.exists("./results/FISH+/EST_stged_reg2.RDS")) {
  stged.est.reg2 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = NULL, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )

  saveRDS(stged.est.reg2, file = "./results/FISH+/EST_stged_reg2.RDS")
} else {
 
  stged.est.reg2 <- readRDS("./results/FISH+/EST_stged_reg2.RDS")
}


reg3 <- TRUE
if (reg3 && !file.exists("./results/FISH+/EST_stged_reg3.RDS")) {
  stged.est.reg3 <- MUR.STged(
    srt_exp = datax$spot_exp, 
    ref_exp = ref_exp, 
    beta.type = beta.type, 
    W = L.mat$weight_adj, 
    lambda1 = 0, 
    lambda2 = 0, 
    cutoff = 0.05, 
    epsilon = 1e-5
  )

  saveRDS(stged.est.reg3, file = "./results/FISH+/EST_stged_reg3.RDS")
} else {
 
  stged.est.reg3 <- readRDS("./results/FISH+/EST_stged_reg3.RDS")
}

We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.1476556 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 


We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.1476556 
Select value of lambda2 0 
Run the main algorithm... 


Select value of lambda1 0 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 


Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 


In [23]:
exp_lsr = st_mu_est(srt_exp = datax$spot_exp,  beta= beta.type )

exp_sc = sc_mu_est( sc_mu = ref_exp, beta = beta.type )

exp_rctd =  RCTDexp(beta = beta.type.RCTD,  srt_exp = datax$spot_exp, ref_mu = ref_exp )

exp_spotdecon = spotdecon_est(srt_exp = datax$spot_exp,  beta= beta.type)


In [24]:
# Check and save exp_ENIGMA result
if (!file.exists("./results/FISH+/EST_exp_ENIGMA.RDS")) {
 exp_ENIGMA = ENIGMA_est(spot_exp  = datax_count$spot_exp, 
 sc_exp = datax_count$sc_exp, sc_label=datax_count$sc_label, 
  beta= NULL)

  # Save the result if computation is performed
  saveRDS(exp_ENIGMA, file = "./results/FISH+/EST_exp_ENIGMA.RDS")
} else {
  # Load the result if it already exists
  exp_ENIGMA <- readRDS("./results/FISH+/EST_exp_ENIGMA.RDS")
}



In [25]:
# Check and save exp_TCA result
if (!file.exists("./results/FISH+/EST_exp_TCA.RDS")) {

  exp_TCA <- TCA_est(spot_exp  = datax_count$spot_exp, beta= beta.type)
  # Save the result if computation is performed
  saveRDS(exp_TCA, file ="./results/FISH+/EST_exp_TCA.RDS")
} else {
  # Load the result if it already exists
  exp_TCA <- readRDS("./results/FISH+/EST_exp_TCA.RDS")
}


In [26]:
est_est= list(STged = stged.est.true$V.hat,
              STged.reg1 = stged.est.reg1$V.hat,
              STged.reg2 =stged.est.reg2$V.hat,
              STged.reg3 =stged.est.reg3$V.hat,
              RCTD = exp_rctd,
              TCA = exp_TCA,
              ENIGMA = exp_ENIGMA,
              ref_mu = exp_sc,
              LSR = exp_lsr,
              Spotdecon = exp_spotdecon)

files = paste0("./results/FISH+/EST_spot_decon.RDS")

saveRDS(est_est, file =files)

Sensitive analysis

In [27]:
beta.type = sim_data$beta.type
beta.type = beta.type[colnames(datax$spot_exp),]

In [28]:
lambda_values1 <- c(0,0.001,0.01,lambda1_opt,1,5,10)
lambda_values2 <- c(0,0.001,0.01,lambda2_opt ,1,5,10)

# Generate all combinations of lambda1 and lambda2 using expand.grid
parameter_grid <- expand.grid(lambda1 = lambda_values1, lambda2 = lambda_values2)

sens_STged = matrix(list(), nrow(parameter_grid) ,1)
for(ii in 1:nrow(parameter_grid)){
  
  cat("Run the STged", "\n")
  
  start_time <- Sys.time()
  stged.est =MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type,   W = L.mat$weight_adj, 
                        lambda1 = parameter_grid[ii,1], lambda2 = parameter_grid[ii,2],
                        cutoff = 0.05, 
                        epsilon = 1e-5)

  end_time <- Sys.time()
  sens_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n")

}

saveRDS(sens_STged, file = "./results/FISH+/True_spot_decon_sens_VAR.RDS")

saveRDS(parameter_grid, file = paste0("./results/FISH+/True_cell_exp_tuning_values_VAR.RDS"))

Run the STged 
Select value of lambda1 0 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 2.405775 
Run the STged 
Select value of lambda1 0.001 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 20.48735 
Run the STged 
Select value of lambda1 0.01 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 23.13423 
Run the STged 
Select value of lambda1 0.03392335 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 17.93713 
Run the STged 
Select value of lambda1 1 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 17.23093 
Run the STged 
Select value of lambda1 5 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 14.76403 
Run the STged 
Select value of lambda1 10 
Select value of lambda2 0 
Run the main algorithm... 
Run time of STged 16.84983 
Run the STged 
Select value of lambda1 0 
Select value of lambda2 0.001 
Run the main algorithm... 
Run time of STg

Benchmark cell type proprotions

In [29]:
res_decon_ct = readRDS("./realdata/FISH+/EnDecon_major/FISH_Results.Deconv.RDS")

In [30]:
dim(res_decon_ct$CARD)

[1] 69  6

In [31]:
methods = names(res_decon_ct)
bench_celltype_STged = matrix(list(), length(methods) ,1)

for(ii in 1: length(methods) ){
  
  cat("Run the STged", methods[ii], "\n")
  beta.type.bench = res_decon_ct[[ii]]
  start_time <- Sys.time()
  stged.est = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type.bench,   W = L.mat$weight_adj, 
                        lambda1 = NULL, lambda2 = NULL, cutoff = 0.05, 
                        epsilon = 1e-5)
                    
  end_time <- Sys.time()
  bench_celltype_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n") 
  
}

files = paste0("./results/FISH+/STged_est_spot_decon_tuning.RDS")

saveRDS(bench_celltype_STged, file =files)

Run the STged CARD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.04661281 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 4.209988 
Run the STged RCTD 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.150627 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 9.309613 
Run the STged SpatialDWLS 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.0350329 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 6.242343 
Run the STged SPOTlight 
We will adpote a value for lambda 1 in our algorithm... 
Select value of lambda1 0.5479117 
tuning for lambda 2 in our algorithm... 
Select value of lambda2 0.1279606 
Run the main algorithm... 
Run time of STged 14.98288 
Run the STged St